In [1]:
from transformers import DetrImageProcessor, DetrForObjectDetection
import torch
from PIL import Image, ImageDraw, ImageFont
import requests
import random


c:\Users\yunus.yesil\Desktop\python-projects\Image_Detection\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [11]:
url = "http://images.cocodataset.org/val2017/000000039769.jpg"
url2 = "https://img-s1.onedio.com/id-558909bed2a15480349e158e/rev-0/w-635/listing/f-jpg-webp/s-7d14de6baa50408719ec71b4ee671a2961a4f81f.webp"
# image = Image.open(requests.get(url2, stream=True).raw)

image = Image.open("IMG_2776.jpg")

# you can specify the revision tag if you don't want the timm dependency
processor = DetrImageProcessor.from_pretrained("facebook/detr-resnet-50", revision="no_timm")
model = DetrForObjectDetection.from_pretrained("facebook/detr-resnet-50", revision="no_timm")

inputs = processor(images=image, return_tensors="pt")
outputs = model(**inputs)

# convert outputs (bounding boxes and class logits) to COCO API
# let's only keep detections with score > 0.9
target_sizes = torch.tensor([image.size[::-1]])
results = processor.post_process_object_detection(outputs, target_sizes=target_sizes, threshold=0.75)[0]

for score, label, box in zip(results["scores"], results["labels"], results["boxes"]):
    box = [round(i, 2) for i in box.tolist()]
    print(
            f"Detected {model.config.id2label[label.item()]} with confidence "
            f"{round(score.item(), 3)} at location {box}"
    )


Detected cup with confidence 0.983 at location [486.47, 1919.56, 755.35, 2297.6]
Detected cup with confidence 0.994 at location [3075.34, 2000.01, 3352.1, 2314.22]
Detected keyboard with confidence 0.999 at location [1413.1, 2112.98, 2396.67, 2340.41]
Detected mouse with confidence 0.999 at location [2575.44, 2143.78, 2832.48, 2331.17]
Detected potted plant with confidence 0.955 at location [2717.97, 1308.36, 3188.37, 1840.3]
Detected cup with confidence 0.963 at location [355.45, 1749.58, 724.94, 2288.0]
Detected tv with confidence 0.998 at location [1214.46, 1169.34, 2772.43, 1820.14]
Detected chair with confidence 0.888 at location [2052.3, 2656.68, 4029.43, 3019.98]
Detected potted plant with confidence 0.905 at location [2855.73, 1523.87, 3190.6, 1842.39]


In [12]:
draw = ImageDraw.Draw(image)

# Define a color mapping dictionary to assign unique colors to each class label
color_map = {}

# Specify font size
font_size = 32
font = ImageFont.truetype("arial.ttf", font_size)

# Specify line width
line_width = 8


img_name = ""
k=0

for score, label, box in zip(results["scores"], results["labels"], results["boxes"]):
    
    box = [round(i, 2) for i in box.tolist()]
    class_label = model.config.id2label[label.item()]
    confidence = round(score.item(), 3)

    # Generate a random color for the class label if it doesn't exist in the color map
    if class_label not in color_map:
        color_map[class_label] = (random.randint(0, 255), random.randint(0, 255), random.randint(0, 255))

    # Draw rectangle with the color corresponding to the class label
    draw.rectangle(box, outline=color_map[class_label], width=line_width)

    # Add text with label and confidence, using the same color as the rectangle
    text = f"{class_label}: {confidence}"
    
    # Create img name
    if k < 4 and class_label not in img_name :
        img_name += "_"+class_label

    draw.text((box[0]+12, box[1]+12), text, fill=color_map[class_label], font=font)
    k += 1

image.show()
image.save(f"img_with{img_name}.png")